In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터셋 로드
food = pd.read_csv('./foodData4_24.csv')

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['RecipeIngredientParts'] = food['RecipeIngredientParts'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['RecipeIngredientParts'])

# 사용자 키워드
# 각 사용자의 키워드를 딕셔너리로 조회
# 아래는 예시 데이터
users = {
    'user1': [['blueberries', 'vanilla yogurt', 'lemon juice', 'granulated sugar', 'milk', 'eggs'],[200]],
    'user3': [['chicken', 'milk', 'eggs', 'Worcestershire sauce', 'salt'], [150]]
}

# 추천 함수 정의
def get_recommendations(user_keywords, tfidf_matrix, food_data):
    # 사용자 키워드를 조회할떼 공백(' ')에 조인하여 하나의 문자열로 처리
    # 사용자 키워드에 대한 TF-IDF 행렬 생성
    user_tfidf_matrix = tfidf.transform([' '.join(user_keywords)])
    
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 음식 추천 알고리즘
    # 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도)을 가져와서
    # 각 음식과의 유사도를 인덱스와 함께 리스트로 변환 => 각 키워드가 가진 유사도를 음식과 전체 조회하여 반환
    sim_scores = list(enumerate(cosine_sim[0]))
   
    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 가장 높은 음식(sim_scores[0])을 제외하고 반환
    # 유사도가 높은 음식으로 경향이 쏠리게 되므로 분위값을 정함
    # 분위값 랜덤으로 적용(numpy) -> 상위값(1:30)제외하고 scores2부터
    # 가장 높은 음식을 반환하는 이유는 검색기록이 1개인 경우 그 데이터가 가장 높은 유사도를 가지게 되므로
    sim_scores1 = sim_scores[1:30]
    sim_scores2 = sim_scores[10001:12000]
    sim_scores3 = sim_scores[20001:22000]
    sim_scores4 = sim_scores[30001:32000]
    sim_scores = sim_scores1 + sim_scores2 + sim_scores3 + sim_scores4

    # sim_scores에 저장된 음식들의 인덱스를 저장
    # sim_scores에 저장된 형태 => (인덱스 번호, 유사도 점수)
    food_indices = [idx[0] for idx in sim_scores]

    # sim_scores에 저장된 음식에 대하여 iloc(food_indices)를 통하여 이름 조회
    return food_data.iloc[food_indices][['Name','Calories']]

###
###  2단계. 조건 추가
###

### 사용자가 섭취 가능한 칼로리로 검색조건 추가

# 사용자가 섭취 가능한 일일 칼로리를 추출
user_calories1 = users['user1'][1][0]
user_calories3 = users['user3'][1][0]

# 하나의 사용자에 대한 음식 추천 실행
keywords1 = users['user1']
print("user1의 유사한 음식 추천:")
recommendations1 = get_recommendations(keywords1[0], tfidf_matrix, food)
print(recommendations1)
print()

keywords3 = users['user3']
print("user3의 유사한 음식 추천:")
recommendations3 = get_recommendations(keywords3[0], tfidf_matrix, food)
print(recommendations3)
print()

# 추천된 음식 중에서 사용자가 섭취 가능한 일일 칼로리 이하의 음식만 필터링
print("user1의 필터된 음식 추천:")
filtered_recommendations1 = recommendations1[recommendations1['Calories'] <= user_calories1]
print(filtered_recommendations1[:20])

print("user3의 필터된 음식 추천:")
filtered_recommendations3 = recommendations3[recommendations3['Calories'] <= user_calories3]
print(filtered_recommendations3)

user1의 유사한 음식 추천:
                                                     Name  Calories
52409                        Blueberry Pineapple Smoothie     259.3
54469                            Apple Blueberry Smoothie     167.4
70067                                Best Blueberry Grunt     428.9
73896                                  Swedish Angel Food     253.7
128228                  Blueberry Sauce, Plain and Simple      67.0
...                                                   ...       ...
54281                                     Garlic Scallops     299.0
115423                   Easy Chocolate Lover's Cheesepie     594.0
155615  Chicken Meatballs With Spinach and Roasted Garlic      64.3
30490                         Kittencal's Famous Coleslaw      92.9
118726                           Cheater's Bananas Foster     429.1

[6026 rows x 2 columns]

user3의 유사한 음식 추천:
                                                     Name  Calories
12211                                  Ham Noodle Muff

In [3]:
# 다양성은 재료, 유저랑 가장 비슷한 경향은 키워드, 카테고리는 애매

In [ ]:
# 재료? 키워드?
# 재료는 한번에 10개 이상의 데이터도 수집 가능
# -> 혼잡도가 높아짐

# 키워드는 재료에 비해 데이터 수집이 느림
# -> 추천범위가 유저의 경향에 쏠리게됨